## 자연어 처리

- https://mangastorytelling.tistory.com/entry/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84%EC%9D%B4%EC%9A%A9%ED%95%9C-%EC%9E%90%EC%97%B0%EC%96%B4-%EC%B2%98%EB%A6%AC-%EC%9E%85%EB%AC%B8-1106-%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%98%81%ED%99%94-%EB%A6%AC%EB%B7%B0-%EA%B0%90%EC%84%B1-%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0Naver-Movie-Review-Sentiment-Analysis

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [37]:
data = pd.read_table('ratings.txt')
data.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [39]:
# 데이터 중복 제거
data = data.dropna(axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199992 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        199992 non-null  int64 
 1   document  199992 non-null  object
 2   label     199992 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [40]:
# 한글과 공백을 제외하고 모두 제거
data['document'] = data['document'].str.replace('[^가-힣\s]','')
data['document'].head()

0                                  어릴때보고 지금다시봐도 재밌어요ㅋㅋ
1    디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...
2                 폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
3    와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...
4                          안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
Name: document, dtype: object

In [41]:
# 불용어 제거 및 데이터 토큰화
from konlpy.tag import Okt

result = []
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords

for sentence in data['document']:
    text = okt.morphs(sentence,stem=True) # 텍스트를 형태소 단위로 나눔
    # stem : 각 단어에서 어간을 추출
    text = [i for i in text if i not in stopwords]
    result += [text]

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(result)
print(tokenizer.word_index)

{'.': 1, '영화': 2, '보다': 3, '..': 4, '을': 5, '...': 6, ',': 7, '없다': 8, '이다': 9, '있다': 10, '좋다': 11, '너무': 12, '?': 13, '다': 14, '정말': 15, '재밌다': 16, '되다': 17, '적': 18, '만': 19, '진짜': 20, '!': 21, '같다': 22, '점': 23, '로': 24, '아니다': 25, '않다': 26, '에서': 27, '만들다': 28, '연기': 29, '나오다': 30, '평점': 31, '것': 32, '최고': 33, '~': 34, '내': 35, '나': 36, '그': 37, '인': 38, '안': 39, '생각': 40, '스토리': 41, '못': 42, '왜': 43, '....': 44, '게': 45, '드라마': 46, '감동': 47, '사람': 48, '1': 49, '보고': 50, '이렇다': 51, '말': 52, '아깝다': 53, '고': 54, '더': 55, '때': 56, '아': 57, 'ㅋㅋ': 58, '배우': 59, '감독': 60, '거': 61, '그냥': 62, '재미있다': 63, '요': 64, '내용': 65, '재미': 66, '중': 67, '까지': 68, '뭐': 69, '주다': 70, '!!': 71, '재미없다': 72, '자다': 73, '하고': 74, '지루하다': 75, '쓰레기': 76, '가다': 77, '시간': 78, '수': 79, '들다': 80, '네': 81, '모르다': 82, '그렇다': 83, '싶다': 84, '사랑': 85, '작품': 86, '지': 87, '10': 88, '알다': 89, '볼': 90, '다시': 91, '하나': 92, '마지막': 93, '이건': 94, '오다': 95, 'ㅋ': 96, '저': 97, '정도': 98, '2': 99, '완전': 100, '많다': 101, 'ㅋㅋㅋ': 102, 

In [43]:
# 등장 빈도수가 3회 미만인 단어들의 분포 확인

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 56705
등장 빈도가 2번 이하인 희귀 단어의 수: 32787
단어 집합에서 희귀 단어의 비율: 57.820298033683095
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.61392158833389


In [44]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 23919


In [45]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(result)
# x = tokenizer.texts_to_sequences(result) # 단어 -> 숫자

In [57]:
word_index = tokenizer.word_index

import json
json = json.dumps(tokenizer.word_index)
f3 = open("wordIndex.json", "w")
f3.write(json)
f3.close()

In [11]:
import numpy as np

y = list(data['label'])

In [12]:
drop_idx = [i for i in range(len(x)) if len(x[i])==0] # 길이가 0인 문자열 제거
len(drop_idx)

379

In [13]:
x = [i for i in x if len(i)!=0]
y = [y[i] for i in range(len(y)) if i not in drop_idx]

In [14]:
print(len(x),len(y))

199613 199613


In [15]:
cnt = 0 
for i in x:
    if len(i) <= 30:
        cnt += 1

print((cnt/len(x))*100) # 길이가 30이하인 리스트의가 전체 리스트의 92.8%

92.84365246752466


In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x = pad_sequences(x,maxlen = 30) # max 길이를 30으로 조정
# 길이가 20이면 그만큼 0으로 채워짐

In [19]:
from tensorflow.keras.layers import Embedding, Dense, LSTM,Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [20]:
x.shape

(199613, 30)

In [30]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [25]:
# 모델 검증

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [26]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(x,np.array(y),test_size=0.2,random_state=42)

In [27]:
print(xtrain.shape)
print(ytrain.shape)

(159690, 30)
(159690,)


In [28]:
xtrain, val_data, ytrain, val_test = train_test_split(xtrain,ytrain,test_size=0.2, random_state=42)
print(xtrain.shape,ytrain.shape)
print(val_data.shape,val_test.shape)

(127752, 30) (127752,)
(31938, 30) (31938,)


In [31]:
# 모델 훈련

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(xtrain, ytrain, epochs=15, callbacks=[es, mc], batch_size=60, validation_data=(val_data,val_test))

Epoch 1/15
2130/2130 [==============================] - ETA: 0s - loss: 0.3972 - acc: 0.8200
Epoch 1: val_acc improved from -inf to 0.83878, saving model to best_model.h5
2130/2130 [==============================] - 208s 95ms/step - loss: 0.3972 - acc: 0.8200 - val_loss: 0.3651 - val_acc: 0.8388
Epoch 2/15


c:\Users\amysm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2130/2130 [==============================] - ETA: 0s - loss: 0.3321 - acc: 0.8585
Epoch 2: val_acc improved from 0.83878 to 0.84749, saving model to best_model.h5
2130/2130 [==============================] - 215s 101ms/step - loss: 0.3321 - acc: 0.8585 - val_loss: 0.3468 - val_acc: 0.8475
Epoch 3/15
2130/2130 [==============================] - ETA: 0s - loss: 0.3020 - acc: 0.8736
Epoch 3: val_acc improved from 0.84749 to 0.85835, saving model to best_model.h5
2130/2130 [==============================] - 198s 93ms/step - loss: 0.3020 - acc: 0.8736 - val_loss: 0.3292 - val_acc: 0.8584
Epoch 4/15
2130/2130 [==============================] - ETA: 0s - loss: 0.2775 - acc: 0.8870
Epoch 4: val_acc improved from 0.85835 to 0.86148, saving model to best_model.h5
2130/2130 [==============================] - 190s 89ms/step - loss: 0.2775 - acc: 0.8870 - val_loss: 0.3277 - val_acc: 0.8615
Epoch 5/15
2130/2130 [==============================] - ETA: 0s - loss: 0.2566 - acc: 0.8981
Epoch 5: val_acc 

In [32]:
evaluate_result = model.evaluate(xtest, ytest)
evaluate_result

1248/1248 [==============================] - 23s 19ms/step - loss: 0.3520 - acc: 0.8579


[0.35202962160110474, 0.8579264879226685]

In [1]:
# 테스트 정확도 측정 -> 모델을 불러올 때 유니코드 에러 발생
from tensorflow.keras.models import load_model

loaded_model = load_model('best_model.h5')

In [1]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [2]:
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
tokenizer = Tokenizer()

import json

# Open the JSON file
with open('for_korean.json', 'r') as f:
    # Read the JSON data and store in word_index
    word_index = json.load(f)

# Now you can use the word_index variable, which contains the data from the JSON file
tokenizer.word_index = word_index

In [3]:
loaded_model = load_model('best_model.h5')

In [4]:
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = 30) # 길이 맞춤
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
      return ("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else:
      return ("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [42]:
sentiment_predict('스트레스로 인해 매우 불안해하고 여유가 없음')

1/1 [==============================] - 0s 65ms/step


'75.49% 확률로 부정 리뷰입니다.'

In [30]:
sentiment_predict('불안함이 완전히 없어지지는 않았지만 마음이 편안해진 것이 보임')

1/1 [==============================] - 0s 56ms/step


'79.41% 확률로 긍정 리뷰입니다.'